In [1]:
#!rm -rf /kaggle/working/*

**NPz file In which we have frames of all video**

In [1]:
!wget "https://drive.usercontent.google.com/download?id=1FozzER12J166m6urMS3dK22ZeymCyzRj&export=download&authuser=0&confirm=t&uuid=8589b68d-51be-45bf-bba7-15ec330e89bf&at=APZUnTV9unkQjv-Tg-asUB7UYRT5%3A1704800955821"

--2024-02-17 03:46:51--  https://drive.usercontent.google.com/download?id=1FozzER12J166m6urMS3dK22ZeymCyzRj&export=download&authuser=0&confirm=t&uuid=8589b68d-51be-45bf-bba7-15ec330e89bf&at=APZUnTV9unkQjv-Tg-asUB7UYRT5%3A1704800955821
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 108.177.125.132, 2404:6800:4008:c01::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|108.177.125.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5595119617 (5.2G) [application/octet-stream]
Saving to: 'download?id=1FozzER12J166m6urMS3dK22ZeymCyzRj&export=download&authuser=0&confirm=t&uuid=8589b68d-51be-45bf-bba7-15ec330e89bf&at=APZUnTV9unkQjv-Tg-asUB7UYRT5:1704800955821.1'

download?id=1FozzER  44%[=======>            ]   2.33G  37.6MB/s    in 55s     


Cannot write to 'download?id=1FozzER12J166m6urMS3dK22ZeymCyzRj&export=download&authuser=0&confirm=t&uuid=8589b68d-51be-45bf-bba7-15ec330e89bf&at=APZUnTV9unkQjv-Tg-asUB7UYRT5:

**Unzip File**

In [2]:
!unzip "/kaggle/working/download?id=1FozzER12J166m6urMS3dK22ZeymCyzRj&export=download&authuser=0&confirm=t&uuid=8589b68d-51be-45bf-bba7-15ec330e89bf&at=APZUnTV9unkQjv-Tg-asUB7UYRT5:1704800955821"

Archive:  /kaggle/working/download?id=1FozzER12J166m6urMS3dK22ZeymCyzRj&export=download&authuser=0&confirm=t&uuid=8589b68d-51be-45bf-bba7-15ec330e89bf&at=APZUnTV9unkQjv-Tg-asUB7UYRT5:1704800955821
   creating: Training Videos/
   creating: Training Videos/preproc_fake_vids/
  inflating: Training Videos/preproc_fake_vids/sa1-video-fadg0.npz  
  inflating: Training Videos/preproc_fake_vids/sa1-video-faks0.npz  
  inflating: Training Videos/preproc_fake_vids/sa1-video-fcft0.npz  
  inflating: Training Videos/preproc_fake_vids/sa1-video-fcmh0.npz  
  inflating: Training Videos/preproc_fake_vids/sa1-video-fdac1.npz  
  inflating: Training Videos/preproc_fake_vids/sa1-video-fdrd1.npz  
  inflating: Training Videos/preproc_fake_vids/sa1-video-fedw0.npz  
  inflating: Training Videos/preproc_fake_vids/sa1-video-felc0.npz  
  inflating: Training Videos/preproc_fake_vids/sa1-video-fjas0.npz  
  inflating: Training Videos/preproc_fake_vids/sa1-video-fjem0.npz  
  inflating: Training Videos/prepro

**Import Libraries**

In [4]:
!pip install mtcnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.9 MB/s eta 0:00:0000:0100:01


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
#from tensorflow.layers.core import Dense
from tensorflow.keras.layers import Flatten, Dense, Input, TimeDistributed, LSTM
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model

In [5]:
import pandas as pd
import pathlib
import cv2
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
from mtcnn import MTCNN
import os
import shutil
import warnings
import time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [6]:
!pwd

/kaggle/working


In [8]:
!wget https://zenodo.org/record/4068245/files/DeepfakeTIMIT.tar.gz?download=1

--2024-02-16 14:16:13--  https://zenodo.org/record/4068245/files/DeepfakeTIMIT.tar.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.185.79.172, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/4068245/files/DeepfakeTIMIT.tar.gz [following]
--2024-02-16 14:16:14--  https://zenodo.org/records/4068245/files/DeepfakeTIMIT.tar.gz
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 226611200 (216M) [application/octet-stream]
Saving to: ‘DeepfakeTIMIT.tar.gz?download=1’

DeepfakeTIMIT.tar.g 100%[===================>] 216.11M  31.3MB/s    in 7.7s    

2024-02-16 14:16:22 (28.0 MB/s) - ‘DeepfakeTIMIT.tar.gz?download=1’ saved [226611200/226611200]



In [9]:
!tar -xf ./DeepfakeTIMIT.tar.gz?download=1

In [10]:
!wget https://lp-prod-resources.s3.amazonaws.com/other/detectingdeepfakes/VidTIMIT.zip

--2024-02-16 14:16:27--  https://lp-prod-resources.s3.amazonaws.com/other/detectingdeepfakes/VidTIMIT.zip
Resolving lp-prod-resources.s3.amazonaws.com (lp-prod-resources.s3.amazonaws.com)... 52.218.221.3, 52.92.138.209, 52.92.209.9, ...
Connecting to lp-prod-resources.s3.amazonaws.com (lp-prod-resources.s3.amazonaws.com)|52.218.221.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1358810924 (1.3G) [application/zip]
Saving to: ‘VidTIMIT.zip’

VidTIMIT.zip        100%[===================>]   1.26G  15.7MB/s    in 80s     

2024-02-16 14:17:47 (16.2 MB/s) - ‘VidTIMIT.zip’ saved [1358810924/1358810924]



In [11]:
!unzip -qq ./VidTIMIT.zip

In [12]:
!rm -rf ./__MACOSX

**Make Directory Of Training and Testing real/fake videos**

In [13]:
!mkdir -p ./Training\ Videos/preproc_real_vids
!mkdir -p ./Training\ Videos/preproc_fake_vids

In [14]:
!mkdir -p ./Testing\ Videos/preproc_real_vids
!mkdir -p ./Testing\ Videos/preproc_fake_vids

In [15]:
!mkdir ./Remaining\ Real\ Videos

In [7]:
real_fake_vids_paths = ["./DeepfakeTIMIT/higher_quality","./VidTIMIT"]

**Counts total number of videos**

In [8]:
for base_vid_path in real_fake_vids_paths:
    
    vid_counter = 0
    for single_vid_path in pathlib.Path(base_vid_path).glob("*/*.avi"):
        
        vid_counter += 1
        
    print("Total Number of Videos are {}".format(vid_counter))
    
    vid_counter = 0

Total Number of Videos are 320
Total Number of Videos are 320


**In Counting of Videos we Found some extra videos so we put them into a Remaining real videos folder**

In [9]:
vid_counter = 0

for single_vid_path in pathlib.Path(real_fake_vids_paths[1]).glob("*/*.avi"):
    
    if str(single_vid_path.parts[-2]) in os.listdir(real_fake_vids_paths[0]):
        vid_counter += 1
        
    else:
        if not os.path.isdir("./Remaining Real Videos/"+single_vid_path.parts[-2]):
            os.mkdir("./Remaining Real Videos/"+single_vid_path.parts[-2])
            
        shutil.move(src=single_vid_path,
                    dst="./Remaining Real Videos/"+"/".join([single_vid_path.parts[-2],
                                                           single_vid_path.parts[-1]]))
        
print("Total Number of Videos are {}".format(vid_counter))

Total Number of Videos are 320


**Now are Total number of videos are equal**

In [10]:
for base_vid_path in real_fake_vids_paths:
    
    vid_counter = 0
    for single_vid_path in pathlib.Path(base_vid_path).glob("*/*.avi"):
        
        vid_counter += 1
        
    print("Total Number of Videos are {}".format(vid_counter))

Total Number of Videos are 320
Total Number of Videos are 320


**Detect Frame**

>In this Function we detect faces in video* 

In [11]:
def detect_face(frame):
    
    face_detector = MTCNN()
    face_attr = face_detector.detect_faces(frame)
    
    if len(face_attr) > 0:
        if len(face_attr[0]['box']) > 0:
            return face_attr[0]['box'], face_attr[0]['keypoints']['left_eye'], face_attr[0]['keypoints']['right_eye']
    else:
        return [None]*3

**Determine rotation angle**
> In this function we calculate the delta_y and delta_x which are the difference between the left eye and right eye

In [12]:
def determine_rotation_angle(left_eye_kp, right_eye_kp):
    
    delta_y = right_eye_kp[1] - left_eye_kp[1]
    delta_x = right_eye_kp[0] - left_eye_kp[0]
    
    return np.arctan(delta_y/delta_x)

In [13]:
def determine_scaling_factor(rescaled_img_size,left_eye_kp,right_eye_kp):
    
    left_eye_kp = np.array(left_eye_kp)
    right_eye_kp = np.array(right_eye_kp)
    
    whole_frame_eye_dist = np.linalg.norm(x=(right_eye_kp - left_eye_kp),ord=2)
    
    cropped_left_eye_kp = np.array([0.35*rescaled_img_size[0], 0.35*rescaled_img_size[1]])
    cropped_right_eye_kp = np.array([0.35*rescaled_img_size[0], 0.65*rescaled_img_size[1]])
    
    cropped_frame_eye_dist = np.linalg.norm(x=(cropped_right_eye_kp - cropped_left_eye_kp),
                                            ord=2)
    
    scaling_factor = cropped_frame_eye_dist/whole_frame_eye_dist
    
    return scaling_factor, cropped_left_eye_kp, cropped_right_eye_kp

In [14]:
def determine_rotation_center(left_eye_kp, right_eye_kp):
    
    center_x = (left_eye_kp[0] + right_eye_kp[0])/2
    center_y = (left_eye_kp[1] + right_eye_kp[1])/2
    
    return (center_x, center_y)

**Prepoc Single Video**
> in this function we crop a faces from the video and convert them into a npz file 

In [25]:
def preproc_single_vid(mb_vids_paths):
    
    for single_vid_path in mb_vids_paths:
        
        print("\n\nGoing to preprocess Video at {}".format(single_vid_path))

        video = cv2.VideoCapture(str(single_vid_path))
        cropped_aligned_faces_list = list()

        while True:

            is_frame, frame = video.read()

            if not is_frame:
                break
            """    
            print("Going to preprocess Frame of {} having size {}".format(single_vid_path.parts[-1],
                                                                  frame.shape))
            """
            face_bbox, left_eye_kp, right_eye_kp = detect_face(frame)
            
            if face_bbox == None:
                continue
                
            rotation_angle = determine_rotation_angle(left_eye_kp, right_eye_kp);
            scaling_factor, cropped_left_eye_kp, cropped_right_eye_kp = determine_scaling_factor((224,224), 
                                                                                            left_eye_kp, right_eye_kp)
            rotation_center = determine_rotation_center(left_eye_kp, right_eye_kp)
            
            #print("Computed everything to get the Rotation Matrix for the Frame")

            rotation_matrix = cv2.getRotationMatrix2D(center=rotation_center,
                                                         angle=rotation_angle,scale=scaling_factor)
            
            #print("Computed the Rotation Matrix for the Frame")

            cropped_center_x = cropped_left_eye_kp[0]
            cropped_center_y = cropped_left_eye_kp[1] + (np.linalg.norm(x=cropped_right_eye_kp-cropped_right_eye_kp,ord=2)/2)
            cropped_center = (cropped_center_x, cropped_center_y)

            rotation_matrix[0,2] = rotation_matrix[0,2] - (rotation_center[0] - cropped_center[0])
            rotation_matrix[1,2] = rotation_matrix[1,2] - (rotation_center[1] - cropped_center[1])
            
            #print("Translated the center in Rotation Matrix to the Cropped Frame Center")

            cropped_aligned_face = cv2.warpAffine(frame,rotation_matrix,(224,224),
                                                      cv2.INTER_CUBIC)
            
            #print("Cropped and Aligned the face in the Frame")

            cropped_aligned_faces_list.append(cropped_aligned_face)
            
            print("Preprocessed Frame of {}".format(single_vid_path.parts[-1]))

        video.release()

        cropped_aligned_faces_list = np.array(cropped_aligned_faces_list)

        if "DeepfakeTIMIT" in list(single_vid_path.parts):
            np.savez("./Training Videos/preproc_fake_vids/{}.npz".format(str(single_vid_path.parts[-1]).replace(".avi","")),
                             args=cropped_aligned_faces_list)
        else:
            np.savez("./Training Videos/preproc_real_vids/{}.npz".format(str(single_vid_path.parts[-1]).replace(".avi","")),
                             args=cropped_aligned_faces_list)

        print("\n\nProcessed Video at {}".format(single_vid_path))

**To start this cell we have to use the GPU so we can't run this cell until and unless you have GPU**

In [ ]:
vid_counter = 0
mb_number = 0

real_fake_preproc_vids_paths = ["./Training Videos/preproc_fake_vids",
                               "./Training Videos/preproc_real_vids"]

for vids_path, preproc_vids_path in zip(real_fake_vids_paths,
                                                    real_fake_preproc_vids_paths):
    
    vids2preproc_path_list = list()
    
    for single_vid_path in pathlib.Path(vids_path).glob("*/*.avi"):
        
        vids2preproc_path_list.append(single_vid_path)
        vid_counter += 1
        
        if vid_counter % (os.cpu_count()) == 0:
            
            start_time = time.time()
            
            with ProcessPoolExecutor(max_workers=(os.cpu_count())) as pool:
                future = pool.submit(preproc_single_vid,vids2preproc_path_list)
                future.result()
                
            end_time = time.time()
            mb_number += 1
            elapsed_time = end_time - start_time
                
            print("\n\nProcessed Mini Batch # {} of 64 Videos in {} seconds".format(mb_number,elapsed_time)) 
                
            vids2preproc_path_list = list()

**Now we split are real and fake videos using hypen **

In [15]:
training_real_vids_paths = list()
training_fake_vids_paths = list()

for single_img_path in pathlib.Path("./Training Videos").glob("*/*.npz"):
    
    if "real" in str(single_img_path.parts[-2]).split("_"):
        training_real_vids_paths.append(str(single_img_path))
    
    elif "fake" in str(single_img_path.parts[-2]).split("_"):
        training_fake_vids_paths.append(str(single_img_path))

In [16]:
training_real_vids_paths

['Training Videos/preproc_real_vids/si2149.npz',
 'Training Videos/preproc_real_vids/si1653.npz',
 'Training Videos/preproc_real_vids/si569.npz',
 'Training Videos/preproc_real_vids/si824.npz',
 'Training Videos/preproc_real_vids/si1199.npz',
 'Training Videos/preproc_real_vids/sx13.npz',
 'Training Videos/preproc_real_vids/si1469.npz',
 'Training Videos/preproc_real_vids/si730.npz',
 'Training Videos/preproc_real_vids/si649.npz',
 'Training Videos/preproc_real_vids/sx98.npz',
 'Training Videos/preproc_real_vids/sx318.npz',
 'Training Videos/preproc_real_vids/si2104.npz',
 'Training Videos/preproc_real_vids/si1099.npz',
 'Training Videos/preproc_real_vids/si1454.npz',
 'Training Videos/preproc_real_vids/sx279.npz',
 'Training Videos/preproc_real_vids/si844.npz',
 'Training Videos/preproc_real_vids/sx95.npz',
 'Training Videos/preproc_real_vids/sx49.npz',
 'Training Videos/preproc_real_vids/si1544.npz',
 'Training Videos/preproc_real_vids/si1669.npz',
 'Training Videos/preproc_real_vids

In [17]:
len(training_real_vids_paths)

197

In [20]:
training_fake_vids_paths

['Training Videos/preproc_fake_vids/sx368-video-fdrd1.npz',
 'Training Videos/preproc_fake_vids/sx29-video-mmdm2.npz',
 'Training Videos/preproc_fake_vids/sa2-video-mgwt0.npz',
 'Training Videos/preproc_fake_vids/sx140-video-fjem0.npz',
 'Training Videos/preproc_fake_vids/sa2-video-fadg0.npz',
 'Training Videos/preproc_fake_vids/si1542-video-mdab0.npz',
 'Training Videos/preproc_fake_vids/si1084-video-faks0.npz',
 'Training Videos/preproc_fake_vids/sa2-video-fedw0.npz',
 'Training Videos/preproc_fake_vids/si2173-video-mcem0.npz',
 'Training Videos/preproc_fake_vids/sa1-video-mstk0.npz',
 'Training Videos/preproc_fake_vids/si911-video-mdbb0.npz',
 'Training Videos/preproc_fake_vids/sx199-video-fram1.npz',
 'Training Videos/preproc_fake_vids/sx209-video-mmdm2.npz',
 'Training Videos/preproc_fake_vids/si1279-video-fram1.npz',
 'Training Videos/preproc_fake_vids/sx205-video-mrcz0.npz',
 'Training Videos/preproc_fake_vids/sx100-video-fadg0.npz',
 'Training Videos/preproc_fake_vids/sa2-video

In [37]:
def custom_training_data_generator(mb_size):
    
    for i in range(len(training_fake_vids_paths)//2):
        
        X_train_mb = list()
        Y_train_mb = list()
        
        training_fake_mb_vids_paths = training_fake_vids_paths[i*(mb_size//2):(i+1)*(mb_size//2)]
        training_real_mb_vids_paths = training_real_vids_paths[i*(mb_size//2):(i+1)*(mb_size//2)]
        
        for single_fake_vid_path,single_real_vid_path in zip(training_fake_mb_vids_paths,
                                                             training_real_mb_vids_paths):
            X_train_mb.append(np.load(file=single_real_vid_path)['args'])
            X_train_mb.append(np.load(file=single_fake_vid_path)['args'])
            
            Y_train_mb.append(0)
            Y_train_mb.append(1)
            
        #print(X_train_mb)
        
        #X_train_mb = np.concatenate(X_train_mb,axis=-1)
        X_train_mb = np.array(X_train_mb, dtype = 'object')
        Y_train_mb = np.array(Y_train_mb)
        
        yield X_train_mb,Y_train_mb

In [126]:
train_datagen = custom_training_data_generator(4)

In [129]:
fg=train_datagen.__next__()

In [137]:
fg[0][0].shape

(122, 224, 224, 3)

In [23]:
data = np.load(r'/kaggle/working/Training Videos/preproc_fake_vids/sa2-video-fjwb0.npz')

**Exploratary Data Analysis**

In [39]:
videos = []
df = data['args']
for i in training_real_vids_paths:
    data = np.load(i)
    videos.append(data['args'])
len(videos)

197

In [ ]:
plt.hist(videos)
plt.title("Frequnecy of videos")

**Padding and Slicing Of Video**
> In this cell we convert all frame into a fixed length size that is 100
>> So we slice video which have more then 100 frames
>>> And Padded That video which have less then 100 frames to repeat last frame of video

In [26]:
from concurrent.futures import ThreadPoolExecutor
import numpy as np

videos = []
df = data['args']
for i in training_real_vids_paths[0:10]:
    data = np.load(i)
    videos.append(data['args'])

def padding(videos):
    return np.concatenate([videos, np.tile(videos[-1], ((100 - videos.shape[0]), 1, 1, 1))])

def slicing(videos):
    return videos[:100]

def frame_equalizer(video):
    num_frames = video.shape[0]  # Get the number of frames in the video
    if num_frames < 100:
        return padding(video)
    elif num_frames > 100:
        return slicing(video)
    else:
        return video

# Apply frame_equalizer function to each video in videos list
result_list = list(map(frame_equalizer, videos))

for idx, result_tensor in enumerate(result_list):
    print(f"Result Tensor {idx + 1} Shape:", result_tensor.shape)


Result Tensor 1 Shape: (100, 224, 224, 3)
Result Tensor 2 Shape: (100, 224, 224, 3)
Result Tensor 3 Shape: (100, 224, 224, 3)
Result Tensor 4 Shape: (100, 224, 224, 3)
Result Tensor 5 Shape: (100, 224, 224, 3)
Result Tensor 6 Shape: (100, 224, 224, 3)
Result Tensor 7 Shape: (100, 224, 224, 3)
Result Tensor 8 Shape: (100, 224, 224, 3)
Result Tensor 9 Shape: (100, 224, 224, 3)
Result Tensor 10 Shape: (100, 224, 224, 3)


In [ ]:
result_list

**Make a Numpy array of same legth of result_list**
> So Now we adding labels in are videos if Videos are real we label them into 1

In [27]:
# Y_train_mb contain all ones 
Y_train_mb = np.ones(10, dtype=int)

In [27]:
type(Y_train_mb)

numpy.ndarray

result_list contain all the real videos

real video is denoted by 1

fake video is denoted by 0

**Convert Them into a DataFrame along With labels**

In [37]:
data = {'video_tensor':result_list,'label':Y_train_mb}

In [38]:
df = pd.DataFrame(data)

**Now We Do same thing on the fake videos**
> padding and slicing 

In [30]:
fake_data = np.load("/kaggle/working/Training Videos/preproc_fake_vids/si1808-video-fdrd1.npz")

In [31]:
fake_data['args'].shape

(113, 224, 224, 3)

In [32]:
from concurrent.futures import ThreadPoolExecutor
import numpy as np

videos = []
df = fake_data['args']
for i in training_fake_vids_paths[0:10]:
    fake_data = np.load(i)
    videos.append(fake_data['args'])

def padding(videos):
    return np.concatenate([videos, np.tile(videos[-1], ((100 - videos.shape[0]), 1, 1, 1))])

def slicing(videos):
    return videos[:100]

def frame_equalizer(video):
    num_frames = video.shape[0]  # Get the number of frames in the video
    if num_frames < 100:
        return padding(video)
    elif num_frames > 100:
        return slicing(video)
    else:
        return video

# Apply frame_equalizer function to each video in videos list
fake_videos = list(map(frame_equalizer, videos))

for idx, result_tensor in enumerate(fake_videos):
    print(f"Result Tensor {idx + 1} Shape:", result_tensor.shape)


Result Tensor 1 Shape: (100, 224, 224, 3)
Result Tensor 2 Shape: (100, 224, 224, 3)
Result Tensor 3 Shape: (100, 224, 224, 3)
Result Tensor 4 Shape: (100, 224, 224, 3)
Result Tensor 5 Shape: (100, 224, 224, 3)
Result Tensor 6 Shape: (100, 224, 224, 3)
Result Tensor 7 Shape: (100, 224, 224, 3)
Result Tensor 8 Shape: (100, 224, 224, 3)
Result Tensor 9 Shape: (100, 224, 224, 3)
Result Tensor 10 Shape: (100, 224, 224, 3)


**Fake video label**
> fake Video labels is 0

In [39]:
fake_video_label=np.zeros((10,), dtype=int)

In [35]:
type(fake_video_label)

numpy.ndarray

**Same as previous  Convert Them into The dataframe**

In [40]:
fake_data = {"video_tensor":fake_videos,"label":fake_video_label}

In [41]:
fake_df = pd.DataFrame(fake_data)

**Now concatenate Both The Dataframe**

In [42]:
videos_data=pd.concat([df,fake_df])

**Take the sample From the Dataset**

In [43]:
videos_data = videos_data.sample(5)

In [44]:
videos_data.shape

(5, 2)

In [176]:
type(videos_data)

pandas.core.frame.DataFrame

**Convert Them into Np.array**
> Now we take Both The columns of the Dataframe and convert them into np.array becoz in model we can't provide a datafame

In [45]:
video_tensors = np.array(videos_data["video_tensor"].tolist())
labels = np.array(videos_data["label"])

In [ ]:
video_tensors[0]

In [82]:
labels[0]

1

**Train And Test Split**

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(video_tensors, labels, test_size=0.2, random_state=42)

# Model
> In this model We use RestNet50
>> ResNet50 is a powerful pretrained network, we use sequnetial model in it.
>>>Create a Sequential model
>>>>Load ResNet50 with specific input shape and exclude top layers
>>>>Set ResNet50 layers as non-trainable
>>>>>Wrap ResNet50 in a TimeDistributed layer
>>>>>>Flatten the output of ResNet50
>>>>>>>Add LSTM layer
>>>>>>>>Add Dense layer with sigmoid activation
>>>>>>>>>Compile the model


In [52]:
model = Sequential()
Res_base = ResNet50(include_top=False, input_shape=(224, 224, 3))
Res_base.trainable = False
model.add(TimeDistributed(Res_base, input_shape=(100, 224, 224, 3))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

**Model Summary**

In [53]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_2 (TimeDi  (None, 100, 7, 7, 2048)   23587712  
 stributed)                                                      
                                                                 
 time_distributed_3 (TimeDi  (None, 100, 100352)       0         
 stributed)                                                      
                                                                 
 lstm_1 (LSTM)               (None, 50)                20080600  
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 43668363 (166.58 MB)
Trainable params: 20080651 (76.60 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


**Model Fit**

In [55]:
model.fit(video_tensors, labels, epochs = 4)

Epoch 1/4
1/1 [==============================] - 63s 63s/step - loss: 0.5248 - accuracy: 0.6000
Epoch 2/4
1/1 [==============================] - 63s 63s/step - loss: 0.5033 - accuracy: 0.6000
Epoch 3/4
1/1 [==============================] - 63s 63s/step - loss: 0.4846 - accuracy: 0.8000
Epoch 4/4
1/1 [==============================] - 62s 62s/step - loss: 0.4810 - accuracy: 0.8000
